This data comes from https://statswales.gov.wales/Catalogue/Community-Safety-and-Social-Inclusion/Welsh-Index-of-Multiple-Deprivation/WIMD-Indicator-Data-By-Age/incomedeprivation

In [11]:
import numpy as np
import pandas as pd

In [15]:
file = 'income_deprivation.csv'
pd.read_csv(file,skiprows=8)

,Unnamed: 0,0 - 15,0 - 18,16 - 64,65+,0 - 4,5 - 9,10 - 15,16 - 18,19 - 24,25 - 34,35 - 44,45 - 54,55 - 64,65 - 74,75+,.
0,Wales,25,24.0,13.0,18.0,27.0,24.0,23.0,21.0,12.0,14.0,13.0,12.0,11.0,14.0,22.0,16.0
1,Isle of Anglesey 001A,31,28.0,19.0,26.0,34.0,30.0,30.0,18.0,21.0,19.0,22.0,17.0,17.0,23.0,30.0,23.0
2,Isle of Anglesey 001B,25,23.0,10.0,10.0,29.0,28.0,21.0,9.0,14.0,13.0,16.0,9.0,6.0,6.0,15.0,12.0
3,Isle of Anglesey 001C,24,22.0,12.0,19.0,31.0,20.0,21.0,13.0,18.0,15.0,9.0,8.0,14.0,15.0,25.0,16.0
4,Isle of Anglesey 001D,22,22.0,12.0,21.0,24.0,23.0,21.0,22.0,15.0,11.0,9.0,10.0,12.0,18.0,25.0,16.0
5,Isle of Anglesey 001E,22,24.0,13.0,15.0,22.0,30.0,16.0,33.0,13.0,18.0,13.0,11.0,9.0,13.0,17.0,15.0
6,Isle of Anglesey 003A,58,52.0,27.0,34.0,69.0,53.0,50.0,28.0,28.0,27.0,35.0,23.0,22.0,36.0,31.0,34.0
7,Isle of Anglesey 003B,50,47.0,20.0,22.0,59.0,42.0,50.0,27.0,28.0,26.0,19.0,17.0,12.0,24.0,19.0,26.0
8,Isle of Anglesey 003C,32,33.0,23.0,26.0,41.0,31.0,25.0,44.0,40.0,20.0,19.0,23.0,20.0,18.0,36.0,26.0
9,Isle of Anglesey 003D,34,34.0,25.0,30.0,42.0,29.0,31.0,39.0,41.0,19.0,25.0,23.0,23.0,31.0,28.0,28.0
